Show how to create a collection and upload data to it but also delete a collection

In [ ]:
import sys, pathlib

repo_root = pathlib.Path().resolve()
repo_root = repo_root.parent

if str(repo_root) not in sys.path:
    sys.path.append(str(repo_root))

from indexing.qdrant import load_qdrant_client, create_qdrant_collection, delete_collection
from indexing.upload import upload_points
from qdrant_client import models
from langchain_qdrant import QdrantVectorStore, RetrievalMode, FastEmbedSparse
from embeddings.embedding import FastEmbedEmbeddings, output_supported_models

In [ ]:
client = load_qdrant_client()

Output possible model

In [ ]:
output_supported_models("dense") 

In [ ]:
output_supported_models("sparse") 

## Create a collection

In [ ]:
model_dense = FastEmbedEmbeddings(model_name="BAAI/bge-base-en-v1.5")
model_sparse = FastEmbedSparse(model_name="Qdrant/bm42-all-minilm-l6-v2-attentions")

In [ ]:
create_qdrant_collection(client, collection_name="RAG", model_dense=model_dense, model_sparse=model_sparse)

For filtering purpose, I need to create a payload

In [ ]:
#Right now, I will only filter on the type but next I will use advanced filtering
#Need to create payload before adding points because it will add the id directly for better filtering
client.create_payload_index(collection_name="RAG", field_name="metadata.type", field_schema=models.PayloadSchemaType.KEYWORD, wait=True)

In [ ]:
vector_store = QdrantVectorStore(client=client, collection_name="RAG", retrieval_mode=RetrievalMode.HYBRID, embedding=model_dense, sparse_embedding=model_sparse)

## Upload points

In [ ]:
upload_points(vector_store)

## Delete collection

In [ ]:
delete_collection(client)